In [1]:
import os
from scipy.spatial.distance import cdist
from sklearn.neighbors import KDTree
from datasets.aflwnet import AFLW2000
from skimage import io, transform
from sklearn.neighbors import KDTree
import scipy.io
import numpy as np

/home/yiwen/anaconda3/envs/3dproject/lib/python3.7/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [4]:
with open('AF_mat_path.txt', 'r') as file1, open('AF_txt_path.txt', 'r') as file2:
    for line1, line2 in zip(file1, file2):
        file_path1 = line1.strip()
        file_path2 = line2.strip()
        mat = scipy.io.loadmat(file_path1)
        points_transposed = np.transpose(mat['pt3d_68'])
        
        face_3ddata = np.loadtxt(file_path2, delimiter=",")
        pts, normals = face_3ddata[:, :3], face_3ddata[:, 3:-1]
        
        tree = KDTree(np.array(pts))
    
        # Define the radii
        radius = [290, 247]

        # Define the weights
        weights = [1, 2]

        # Create an empty weights array
        weights_array = np.zeros(np.array(pts.shape)[0])
    

        # Iterate over the reference points
        for point_transposed in points_transposed:
            point_of_interest = (point_transposed /1000.)
            for i in range(2):
                # Query the KDTree for the indices of the points within the specified radius
                indices = tree.query_radius(point_of_interest.reshape(1, -1), r=radius[i])

                # Assign the corresponding weight to the selected points
                weights_array[indices[0]] = weights[i]   
        # print(points_weight.shape)
        points_weight = np.hstack((pts, normals, weights_array.reshape(-1, 1)))
        # print(normals.shape)
        if os.path.exists(file_path2):
            os.remove(file_path2)
        np.savetxt(file_path2, points_weight,delimiter=",", fmt="%.6f")